### Open sampler

In [1]:
from src.utils import open_sampler, generate_password

password = generate_password(8)
print(password)

open_sampler(username='edenfatachov',
            password=password,
            phone='+9720522808159'
)


iIocXGjG
<Response [200]>
201 {'username': 's_edenfatachov'}


### Open user

In [1]:
from src.utils import generate_digits_password, open_patient, hash_phone_number
from src.settings import Settings
import pandas as pd

open_patient(phone="+330612444491",
             password=generate_digits_password(),
             birth_date="1965-12-12",
             gender='female',
             mother_tongue='hebrew',
             year_of_diagnosis=2021,
             smoking_routine='past_smoker',
             genetic="NO",
             language='he',
             sampler_phone="+9720549776075",
             demo=False,
)



201 {'username': '9352f14b777f10671e0920773285186d45e5ea1a'}


### Healthy
* Read the csv of the new users.
* Compare with logs: if there are new HC users, append them to the csv; use all the columns in the csv.
* Add new columns: 
    * opened: datetime
    * recorded: bool
    * reminder1: datetime
    * reminder2: datetime
    * reminder3: datetime
    * deleted: datetime
* Open newly registered participants (with opened=False)
* List all active HC users:
    * If a user was opened but no longer active: recorded=True
    * else, look for reminders- 
        * if (1) is empty and it has been a week since registration, send a reminder and doc timestamp.
        * else, compare with last reminder.
        * else (more than a week since reminder 3): delete user and doc timestamp.
* For regular users do not engage. 

In [ ]:
from src.query import list_bucket_for_user
from src.utils import hash_phone_number as hp, open_healthy, delete_user
from src.patterns import Patterns, extract_from_filename
from src.settings import Settings
from src.send import send_hc_weekly_reminder
from datetime import datetime, date
import re
import pandas as pd


daysSinceLastSMS = 6

def check_when_hc_recorded(phone: str) -> str:
    """
    Check when a user with a hashed phone number has recorded the latest "feedback" exercise.
    If the "feedback" file's date is the latest among all files, return its full timestamp. Otherwise, return None.
    """
    # Generate the hashed username
    if len(phone)<40:
        username = f"hc_{hp(f'+972{phone}')}"
    else: 
        username = phone
    
    # List all files in the bucket for the user
    file_keys = list_bucket_for_user(username)
    if not file_keys:
        return None
    # Initialize variables to track the latest feedback and absolute latest date
    latest_feedback_timestamp = None
    latest_feedback_date = None
    absolute_latest_date = None
    
    for file_key in file_keys:
        # Extract the filename from the file key
        filename = file_key.split("/")[-1]

        # Extract the datetime from the filename
        file_timestamp_str = extract_from_filename(filename, 'datetime')
        if not file_timestamp_str:
            continue  # Skip files that do not match the expected timestamp pattern
        
        file_datetime = datetime.strptime(file_timestamp_str, Settings.DATETIME)
        file_date = file_datetime.date()  # Extract the date for comparison

        # Update the absolute latest date
        if absolute_latest_date is None or file_date > absolute_latest_date:
            absolute_latest_date = file_date

        # Check if the file matches the RECORDING1 pattern
        if re.match(Patterns.RECORDING, filename):
            # Extract the exercise type
            exercise = extract_from_filename(filename, 'exercise')
            if exercise == "feedback":
                # Update the latest feedback date and timestamp
                if latest_feedback_date is None or file_date > latest_feedback_date:
                    latest_feedback_date = file_date
                    latest_feedback_timestamp = file_datetime

    # Return the feedback timestamp only if its date matches the absolute latest date
    if latest_feedback_date == absolute_latest_date:
        return latest_feedback_timestamp.strftime(Settings.FORM_DATETIME)
    return None


def user_dict_from_row(row):
    user_dict = {}
    user_dict['healthy_name'] = row['שם מלא']
    user_dict['send_to'] = f"+972{row['טלפון']}"
    
    dob = f"{row['שנת לידה']}-01-01"
    date_object = date.fromisoformat(dob)
    user_dict['birth_date'] = date_object.isoformat()

    user_dict['gender'] = 'male' if row['מגדר']=='גבר' else 'female'
    user_dict['smoking_routine'] = 'smoker' if row['שאלות כלליות [מעשן/ת?]']=='כן' else 'non_smoker'
    user_dict['mother_tongue'] = row['שפת אם'][:-3]
    user_dict['sleep_talk'] = True if row['שאלות כלליות [האם את/ה זז/ה הרבה מתוך שינה?]']=='כן' else False
    user_dict['constipation'] = True if row['שאלות כלליות [האם את/ה סובל/ת מעצירות?]']=='כן' else False
    user_dict['falling'] = True if row['שאלות כלליות [האם נפלת יותר מ 3 פעמים במהלך השנה האחרונה?]']=='כן' else False
    user_dict['smell'] = True if row['שאלות כלליות [האם יש ירידה בחוש הריח?]']=='כן' else False
    genetic = row['האם יש לך נטייה גנטית לפרקינסון?']
    if (genetic=='GBA') or (genetic=='LRRK2'):
        user_dict['genetic']=genetic
    elif genetic=='לא ידוע לי':
        user_dict['genetic']='NO'
    else:
        user_dict['genetic']='YES'
    return user_dict



users_df = pd.read_csv(Settings.FORM_CSV, dtype=str)
users_log = pd.read_csv(Settings.OPEN_HEALTHY, dtype=str)

# log and open new HC users
# registered_users = users_df[users_df['השתתפות בתור']=='בריא/ה ללא אבחנה של מחלה נוירודגנרטיבית כלשהי']['טלפון'].tolist()
logged_users = users_log['טלפון'].tolist()
new_users = users_df[
    (users_df['השתתפות בתור'] == 'בריא/ה ללא אבחנה של מחלה נוירודגנרטיבית כלשהי') &
    (~users_df['טלפון'].isin(logged_users))
].copy()

for col in users_log.columns:
    if col not in new_users.columns:
        new_users[col] = None  # Add missing columns with None values

new_users = new_users[users_log.columns]
updated_users_log = pd.concat([users_log, new_users], ignore_index=True)

# Iterate over all logged users
for idx, user in updated_users_log.iterrows():
    if pd.isna(user['opened']) or user['opened'] == '':
        # Open account for new user
        user_dict = user_dict_from_row(user)
        timestamp = datetime.now().strftime(Settings.FORM_DATETIME)
        response = open_healthy(user_dict, sampler='+9720532752715')
        if response.status_code == 201:
            print(f"User {user_dict['send_to']} opened successfully")
            updated_users_log.at[idx, 'opened'] = timestamp
        else:
           print(response)         
        continue
    else:
        # Check if the user has recorded or deleted
        if not pd.isna(user['recorded']) and user['recorded'] != '':
            continue  # User has already recorded, move to the next
        if not pd.isna(user['deleted']) and user['deleted'] != '':
            continue  # User has been deleted, move to the next
        
        # Check recording status
        recording_timestamp = check_when_hc_recorded(user['טלפון'])
        if recording_timestamp:
            updated_users_log.at[idx, 'recorded'] = recording_timestamp
            continue
        
        # Handle reminders and deletion
        opened_date = datetime.strptime(user['opened'], Settings.FORM_DATETIME)
        today = datetime.now()
        days_since_opened = (today - opened_date).days
        
        for reminder_col, prev_reminder_col in [('reminder1', 'opened'), ('reminder2', 'reminder1'), ('reminder3', 'reminder2')]:
            if pd.isna(user[reminder_col]) or user[reminder_col] == '':
                prev_date = datetime.strptime(user[prev_reminder_col], Settings.FORM_DATETIME)
                if (today - prev_date).days >= daysSinceLastSMS:
                    timestamp = datetime.now().strftime(Settings.FORM_DATETIME)
                    send_hc_weekly_reminder(user['טלפון'])
                    updated_users_log.at[idx, reminder_col] = timestamp
                break  # Exit loop after sending a reminder
        else:
            last_reminder_date = datetime.strptime(user['reminder3'], Settings.FORM_DATETIME)
            days_since_last_reminder = (today - last_reminder_date).days
            print(f"Deleting user {user['טלפון']} - Days since last reminder: {days_since_last_reminder}")
            delete_user(user)
            timestamp = datetime.now().strftime(Settings.FORM_DATETIME)
            updated_users_log.at[idx, 'deleted'] = timestamp

updated_users_log.to_csv(Settings.OPEN_HEALTHY, index=False)



Listing files for user: hc_f66472f757784554d6c4e464ea8bc7723350df15 from AWS bucket...


100%|██████████| 30/30 [00:00<?, ?it/s]


Recording Notification Message sent to 0549776075 with SID: MMa55822c52b83e1c0255a8d0b5bbd5115

Listing files for user: hc_e15312a3dc1ea42dc9a937d0085e1e8a411eb882 from AWS bucket...


100%|██████████| 1/1 [00:00<?, ?it/s]


Recording Notification Message sent to 0544224936 with SID: MMf694d8f37be48d08ce65d77e5136378e

Listing files for user: hc_de3592fef0be06d8652038cc2446cbec05caf63c from AWS bucket...


...


Listing files for user: hc_2b732dafe67138a9aad97b55e110b5bb80dec226 from AWS bucket...


100%|██████████| 1/1 [00:00<?, ?it/s]



Listing files for user: hc_a33eccaa7bd5c89c5ff6c1493ffa965ab3e79573 from AWS bucket...


100%|██████████| 1/1 [00:00<?, ?it/s]


<Response [409]>


### Get password

In [6]:
from src.utils import get_password

# p = get_password('f0c327e841a9cd65f5f937058de58fc2bcf6bccb'); print(p)
# p = get_password('9720548181082'); print(p)
# p = get_password('0504547258'); print(p)

p = get_password('9720544656771'); print(p)

[{'username': '541b2077093320afcbd6e958482356d7f8b47be9', 'user_phone': '+9720544656771', 'password': '0862276266'}]


### Change password

In [1]:
from src.utils import generate_digits_password, open_demo_patient, hash_phone_number, delete_user, get_password
from src.process_bucket import list_bucket
from src.patterns import Patterns, extract_from_filename
from src.settings import Settings
import re
from datetime import datetime, date
from typing import List
import os
from os.path import join
import pandas as pd
import csv

user_phone = "+9720544674000"
p = get_password(user_phone); print(p)

delete_user(hash_phone_number(user_phone))
pw = generate_digits_password()
open_demo_patient(phone=user_phone, password=pw, sampler_phone='+9720532752715')
files = list_bucket()

def find_latest_registration_file(user_files: List[str], username: str):
    latest_file = None
    latest_datetime = None
    for file in user_files:
        if username not in file:
            continue
        else:
            file = file.split("/")[-1]
        if re.search(Patterns.REGISTRATION, file):
            file_datetime = extract_from_filename(file, 'datetime')
            file_datetime = datetime.strptime(file_datetime, Settings.DATETIME)
            if latest_datetime is None or file_datetime > latest_datetime:
                latest_datetime = file_datetime
                latest_file = file
    filekey = join(username, latest_file)
    return {'category': 'file', 'username': username, 'entry': filekey}

filekey_dict = find_latest_registration_file(files, hash_phone_number(user_phone))
print(f"Filekey {filekey_dict['entry']} was added to Blacklist")
blacklist = pd.read_csv(Settings.BLACKLIST, dtype=str)
blacklist = pd.concat([blacklist, pd.DataFrame([filekey_dict])], ignore_index=True)
blacklist.to_csv(Settings.BLACKLIST, index=False)

with open(Settings.CHANGED_PASSWORDS, 'a', newline='') as csvfile:
    fieldnames = ['user_phone', 'username', 'old_password', 'password', 'date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header if the file is new
    if not os.path.isfile(Settings.CHANGED_PASSWORDS):
        writer.writeheader()
    
    # Append the new row
    writer.writerow({
        'user_phone': user_phone,
        'username': hash_phone_number(user_phone),
        'old_password': p[0]['password'],
        'password': pw,
        'date': date.today().isoformat(),
    })


[{'username': '37b8be5eab3168f2f82ab70b2562872fc5e35a76', 'user_phone': '+9720544674000', 'password': '&cdPquDQK-'}]
User 37b8be5eab3168f2f82ab70b2562872fc5e35a76 deleted successfully
<Response [200]>
201 {'username': '37b8be5eab3168f2f82ab70b2562872fc5e35a76'}


Listing files chunks from AWS bucket: 33it [00:09,  3.33it/s]                      

Filekey 37b8be5eab3168f2f82ab70b2562872fc5e35a76\user_register_2024-12-16_09:02:49.csv was added to Blacklist


### Send a reminder

In [8]:
import pandas as pd
from src.settings import Settings
from src.send import send_recording_notification
from src.utils import get_password


user_phone = '+9720546785867'

send_recording_notification(destination_phone_number=user_phone,
                            username=user_phone[4:],
                            password=get_password(user_phone)[0]['password'])


Recording Notification Message sent to +9720546785867 with SID: MMe11c2fedf475d3007cd06622b0a6b8c6


### Send request for details to healthy

In [2]:
from src.send import send_request_for_details_to_healthy

HC_phones = [
    '+9720549776075',
]

for hc in HC_phones:
    send_request_for_details_to_healthy(hc)

Recording Notification Message sent to +9720549776075 with SID: MM32b5336ad8794a535c1829d4c5dd5eee


### Open Healthy from csv

In [12]:
from src.settings import Settings
from src.utils import open_healthy, user_dict_from_row, hash_phone_number
import pandas as pd
from datetime import date
import time 


path = 'log/source/hc/הצטרפות בריאים - Form Responses 1.csv'

df = pd.read_csv(path, dtype=str)
openhc = pd.read_csv(Settings.OPENED_HC, dtype=str)
opend_phones = openhc['user_phone'].tolist()

for ii,row in df.iterrows():
    user_dict = user_dict_from_row(row)
    if user_dict['send_to'] not in opend_phones:
        # print(f"Phone number {user_dict['send_to']} is new")
        response = open_healthy(user_dict, sampler='+9720549776075')
        # print(user_dict)
        if response.status_code == 201:
            opened_dict = {'user_phone': user_dict['send_to'], 
                            'username': f"hc_{hash_phone_number(user_dict['send_to'])}", 
                            'date': date.today().strftime(Settings.DATE_FORMAT)}
            toappend = pd.DataFrame([opened_dict])
            openhc = pd.concat([openhc, toappend])
            print(f"User {user_dict['send_to']} opened successfully")
        else:
           print(response) 
    else:
        print("User is already open")
    time.sleep(5)
openhc.to_csv(Settings.OPENED_HC, index=False)


User was opened
User was opened
<Response [409]>


In [9]:
opened_dict = {'user_phone': user_dict['send_to'], 
                            'username': f"hc_{hash(user_dict['send_to'])}", 
                            'date': date.today().strftime(Settings.DATE_FORMAT)}
toappend = pd.DataFrame([opened_dict])
openhc = pd.concat([openhc, toappend])
openhc

,user_phone,username,date
0,+9720549776075,hc_f66472f757784554d6c4e464ea8bc7723350df15,2024-10-15


In [5]:
from src.settings import Settings
from src.utils import open_healthy, user_dict_from_row, hash_phone_number, get_user
import pandas as pd
from datetime import date


username =  hash_phone_number('+9720544656371')
get_user(username)

200 {'username': '33c222ca32238b76b58a937dfd6b82f87a61badf'}


### Yom Kipur

In [ ]:
import pandas as pd
from src.settings import Settings
from src.send import yom_kipur


df = pd.read_csv(Settings.SEND_PASSWORDS, dtype=str)
for ii,row in df.iterrows():
    if row['SMS']=='1':
        yom_kipur(destination_phone_number=row['user_phone'])

yom_kipur(destination_phone_number='+9720542050599')
yom_kipur(destination_phone_number='+9720549776075')
yom_kipur(destination_phone_number='+9720532752715')


### Delete users

In [4]:
from src.utils import delete_user
from src.utils import hash_phone_number as hash
from tqdm import tqdm

duser = 'hc_' + hash("+9720549776075")
delete_user(duser)

User hc_f66472f757784554d6c4e464ea8bc7723350df15 deleted successfully


### Hash phone number

In [7]:
from src.utils import hash_phone_number
phones = ['+9720523503473',
'+9720525013968',
'+9720528722548',
'+9720528022830',
'+9720544933416',]

for p in phones:
    hh = hash_phone_number(p)
    print(hh)


d7a8394b62fa97832bbe5a3249ba2c4d19cf9c83
413ea12275b9cc5c0ae32541758aa10c9c8f2022
0ee4b0a00dc60261543c0186130ca9d5b7971846
45081dd3bf9009d73fe8dd37238198b013442651
753d6faf042ac97bc3d1bd1e309fb002b2534cca


# Caregivers

In [2]:
import pandas as pd
from src.settings import Settings
from src.send import send_healthy_notification

badH = pd.read_csv(Settings.SESSIONS, dtype=str)
badH['date'] = pd.to_datetime(badH['date'])
badH = badH[badH['date'] > pd.to_datetime('2024-06-01')]

badH = badH[(badH.entity=='HC') & (badH.user_paradigm.str.contains("~"))]
badH = badH.drop_duplicates(subset=['username'])
badH


,date,entity,username,session,number_of_session,start,end,duration,sampler_username,sampler_paradigm,user_paradigm,caregiver_username,caregiver_session,caregiver_paradigm,caregiver_phone,user_phone,password
16,2024-07-11,HC,hc_48022c9a8fa56d68ca3ec8c28c98b541b0f3dc95,ae0f7ee1-0bb9-413f-826e-994dcc28f5fb,2,15:39:43,15:41:05,0:01:22,s_nitzaramon,NaN,[HHHHHHH~~~~~~~~~~~~~~~~],NaN,NaN,NaN,NaN,9720523317731,8MMEm&_lS6
